# EMNIST

In [10]:
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import transforms , datasets
from torch.utils.data import DataLoader

In [11]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])

In [12]:
train_dataset = datasets.QMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.QMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


100%|██████████| 9.70M/9.70M [00:00<00:00, 170MB/s]
100%|██████████| 463k/463k [00:00<00:00, 15.2MB/s]
100%|██████████| 9.74M/9.74M [00:00<00:00, 135MB/s]
100%|██████████| 527k/527k [00:00<00:00, 15.8MB/s]


In [13]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1,6, kernel_size=5, stride=1, padding=1)
    self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1)
    self.fc1 = nn.Linear(16*4*4, 120) # Corrected input size
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84,10)

  def forward(self, x):
    x = torch.relu(self.conv1(x))
    x = torch.max_pool2d(x , kernel_size=2, stride=2)
    x = torch.relu(self.conv2(x))
    x = torch.max_pool2d(x , kernel_size=2, stride=2)
    x = x.view(-1, 16 * 4 * 4) # Corrected flattening size
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [14]:
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [15]:
def train(model, train_loader ,criteriion, optimizer, epoch):
  model.train()
  for batch_idx , (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    outputs = model(data)
    loss = criteriion(outputs, target)
    loss.backward()
    optimizer.step()

    if batch_idx % 100 == 0:
      print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}")

In [16]:
def test(model, test_loader, criterion):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      outputs = model(data)
      test_loss += criterion(outputs, target).item()

      pred = outputs.argmax(dim=1, keepdim=True)
      correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(test_loader.dataset)
  print(f"\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n")

In [17]:
num_epoch = 25
for epoch in range(1, num_epoch+1):
  train(model, train_loader, criterion, optimizer, epoch)
  test(model, test_loader, criterion)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.312301
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.520458
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.243973
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.255340
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.225784
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.140613
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.056044
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.091558
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.051244
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.122212

Test set: Average loss: 0.0015, Accuracy: 58166/60000 (97%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.187735
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.062144
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.053865
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.040629
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.048662
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.126438
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.071371
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.066574
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.046661
